# Data Exploration Notebook

This notebook demonstrates how to connect to the project's PostgreSQL warehouse and fetch sample data from the `marts` schema for analysis.

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASSWORD = os.getenv("DB_PASSWORD", "password")
DB_NAME = os.getenv("DB_NAME", "medical_warehouse")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

## 1. Fetching Channel Metadata

Let's see what channels are currently in our warehouse.

In [3]:
query_channels = "SELECT * FROM public_marts.dim_channels LIMIT 10"
df_channels = pd.read_sql(query_channels, engine)
df_channels

,channel_key,channel_name,channel_type,first_post_date,last_post_date,total_posts,avg_views
0,2288b595-14f5-4ca9-c72b-1a82fa3d7782,tikvahpharma,Pharmaceutical,2026-01-16,2026-01-21,109,3268.28
1,a22689d1-8ece-d39c-fa39-810a8eef9f50,lobelia4cosmetics,Cosmetics,2026-01-16,2026-01-21,105,354.44
2,0da01218-50fa-d876-e43e-518319784893,CheMed123,Pharmaceutical,2022-09-05,2023-02-10,72,1497.56


## 2. Fetching Sample Messages

Let's fetch a few recently processed messages.

In [5]:
query_messages = """
SELECT 
    message_id, 
    channel_name, 
    message_text, 
    message_datetime, 
    view_count
FROM public_marts.fct_messages 
ORDER BY message_datetime DESC 
LIMIT 5
"""
df_messages = pd.read_sql(query_messages, engine)
df_messages

,message_id,channel_name,message_text,message_datetime,view_count
0,189084,tikvahpharma,**የቻናል ግብዣ**:\n(ለፋርማ ማከፋፈያዎች ብቻ) \n\nየመድኃኒትና የ...,2026-01-21 16:51:23,1
1,189083,tikvahpharma,📌 **Vacancy Announcement:-**\n\n\n**Organizati...,2026-01-21 16:47:04,36
2,22994,lobelia4cosmetics,**APTAMIL NO.3\nPrice 5000 birr \nTelegram :-*...,2026-01-21 14:56:20,115
3,22993,lobelia4cosmetics,APTAMIL NO.1**\nPrice 5000 birr \nTelegram :-*...,2026-01-21 14:56:19,114
4,189082,tikvahpharma,21**-01-2**6\nCurrently available Drugs \n\n\n...,2026-01-21 14:18:58,197


## 3. Image Detection Insights

Summarizing the most frequent objects detected by YOLOv8.

In [7]:
query_detections = """
SELECT 
    detected_class, 
    COUNT(*) as count 
FROM public_marts.fct_image_detections 
GROUP BY detected_class 
ORDER BY count DESC
"""
df_detections = pd.read_sql(query_detections, engine)
df_detections

,detected_class,count
0,none,65
1,bottle,56
2,person,48
3,refrigerator,8
4,book,6
5,cell phone,6
6,parking meter,5
7,clock,5
8,orange,4
9,stop sign,3
